In [2]:
import os
os.chdir('..')

from analysis.fidelity_prediction import FidelityModel
from baselines.fidelity_prediction.rb_prediction import RBModel
from simulator.gate_error_model import GateErrorModel

from analysis.vectorization import RandomwalkModel

from data_objects.random_circuit import random_circuits, random_circuit
from data_objects.backend import Backend, LinearBackend, GridBackend, FullyConnectedBackend

from simulator.noisy_simulator import NoisySimulator
import random
from dataset import load_dataset
from tools.ray_func import map

from data_objects.circuit import SeperatableCircuit

from tools.ray_func import map
import numpy as np
import ray

from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import random

ray.init()


/Users/siwei/miniforge3/envs/janusq/lib/python3.11/subprocess.py:1883: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
2024-04-08 11:17:27,512	INFO worker.py:1724 -- Started a local Ray instance.


Python version:,3.11.7
Ray version:,2.9.2


# Fidelity Prediction Of JanusQ-CT on Real Quantum Device

**Author:** Congliang Lang \& Siwei Tan  

**Date:** 7/4/2024

Based on "[QuCT: A Framework for Analyzing Quantum Circuit by Extracting Contextual and Topological Features (MICRO 2023][1]"

[1]: https://scholar.google.com/scholar_url?url=https://dl.acm.org/doi/abs/10.1145/3613424.3614274%3Fcasa_token%3DffjIB1hQ4ZwAAAAA:8MajDLrDOC74WoeMf7r7AoQ-koxCa4E1TNqQg3GSDz03xUX6XdE3toNTM-YdM_e4rKEusMceJ6BGJg&hl=zh-CN&sa=T&oi=gsb&ct=res&cd=0&d=11146218754516883150&ei=42YSZpPlFL6s6rQPtt6x6Ac&scisig=AFWwaeYaiu2hyx8HUJ_7Buf9Mwom


In this notebook, we use the fidelity dataset of a custom superconducting quantum devices to evaluate the fidelity prediction model of Janus-CT. This device is custom with 18 Xmon qubits arranged in a 6×3 grid qubit topology and use RX, RY, RZ, and CZ gates as basis gates, with gate times of 30 ns and 60 ns for single-qubit and two-qubit gates, respectively. The single-qubit gate fidelity, two-qubit fidelity and measurement are 99.97\%, 99.16\% and 94.91\%, repsecitvely, are benchmarked by isolated RB. For simultaneous RB, the single-qubit and two-qubit fidelities of both devices are above 99\% and 98\%, respectively.

<div style="text-align:center;">
    <img src="pictures/2-3.processor_topology.png"  width="20%" height="20%">
</div>

## Load fidelity dataset

In [6]:
import pickle
with open("./dataset/dataset_20230321_2500.pkl", "rb") as f:
    circuits: list[SeperatableCircuit] = pickle.load(f)   # TODO: 整理成circuits一个数组，fidelity一个数组的形式
print(len(circuits))

2500


A large amount of zero-valued fidelities means less valid information, which negatively affects the training convergence and the model accuracy. For large-scale quantum device, e.g., 18-qubit device, we use seperable circuits to pervent the final fidelity from vanishing to zero due to a large number of gates. To address this, the separable circuits used in the fidelity dataset restrict the entangled qubits into sub-circuits within a small number of qubits.

<div style="text-align:center;">
    <img src="pictures/2-3.lagre_fidelity.png"  width="40%" height="40%">
</div>

In [12]:
sub_circuits, sub_fidelities = [], []
for circuit in circuits:
    for sub_cir in circuit.seperatable_circuits:
        sub_circuits.append(sub_cir)
        sub_fidelities.append(sub_cir.ground_truth_fidelity)

print(len(sub_circuits), len(sub_fidelities))

10000 10000


In [ ]:
# generate train datset and test dataset

## Build Vectorization Model

To train the fidelity prediction model, we first build the vectorization model for the quanutm device.

In [11]:
n_qubits = 18
n_steps = 2
n_walks = 20
backend = GridBackend(3, 6)

vec_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
vec_model.train(circuits, True, remove_redundancy = False)

100%|██████████| 10/10 [01:28<00:00,  8.89s/it]


[array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int8),
 array([[0, 1, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
 

## Train Fidelity Model

In [13]:
fidelity_model = FidelityModel(vec_model)
fidelity_model.train((sub_circuits, sub_fidelities))

8000it [00:00, 1007186.91it/s]
100%|██████████| 8/8 [00:25<00:00,  3.25s/it]


(_map pid=22104) /Users/siwei/workspace/JanusQ/janusq/analysis/fidelity_prediction.py:85: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
(_map pid=22104)   return jnp.array(vec_model.vectorize(circuit), jnp.float64)
(raylet) Spilled 2973 MiB, 15 objects, write throughput 424 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(_map pid=22105) /Users/siwei/workspace/JanusQ/janusq/analysis/fidelity_prediction.py:85: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#cu

: 

## Compare to Random Benchmarking

In [ ]:
# build a random benchmarking model


def plot_scater(reals, predicts, durations ,name):
    par = np.polyfit(reals, predicts, 1, full=True)
    slope=par[0][0]
    intercept=par[0][1]
    x1 = [0.4, 1.0]
    y1 = [slope*xx + intercept  for xx in x1]
    colors = ["#FF3636", '#277C8E' ,"#1F77B4"]
    pos = [0, .5, 1]
    cmap = LinearSegmentedColormap.from_list('my_colormap', list(zip(pos, colors)))

    normalied_durations = (durations - durations.min())/(durations.max() - durations.min())


    random_index = list(range(len(reals)))
    random.shuffle(random_index)
    random_index = random_index[:1500]
    reals = np.array(reals)
    predicts = np.array(predicts)
    fig, axes = plt.subplots(figsize=(10, 10))  # 创建一个图形对象和一个子图对象
    axes.axis([0, 1, 0, 1])
    axes.scatter(reals[random_index], predicts[random_index], c= normalied_durations[random_index], cmap=cmap,alpha = 0.6, s=80 )
    axes.plot(x1,y1)
    axes.set_xlim(.2, 1)
    axes.set_ylim(.2, 1)
    axes.set_xlabel('real ')
    axes.set_ylabel('predict')
    axes.plot([[0,0],[1,1]])
    # fig.colorbar(cm.ScalarMappable( cmap=cmap))
    fig.savefig(name)
    print(slope, intercept)
    
# TODO: RB 没有写完